In [1]:
import mewpy.solvers
from mewpy.omics.expression import ExpressionSet
import numpy as np
import pandas as pd
from src.integration import *
from mewpy import *
from mewpy.omics.integration.gimme import GIMME


Set parameter Username
Academic license - for non-commercial use only - expires 2022-12-11


At this phase the data is ready to be integrated into a metabolic model. To do so, MewPy will be used, which is a Python library that allows to integrate data into a metabolic model. Two methods were implemented: GIMME and eFlux. To run both of them the gene expression dataset will be necessary TPM file to be integrated in a in house GEM.

Load your model. It will give you a brief description of the model, and also understand if the model was properly loaded.

In [2]:
from src.integration import Integration

my_model = Integration(model="data/inputs/model_ngaditana.xml")

72520 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Model loaded
Objective
1.0 e_Biomass__cytop = 1904.5048036884596

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra 7.067E+04         0   0.00%
C00009__extr EX_C00009__dra      2499         0   0.00%
C00011__extr EX_C00011__dra 9.842E+04         1 100.00%
C00059__extr EX_C00059__dra     177.3         0   0.00%
C00080__extr EX_C00080__dra      2676         0   0.00%
C00205__extr EX_C00205__dra     1E+06         0   0.00%
C00244__extr EX_C00244__dra 1.611E+04         0   0.00%
C00305__extr EX_C00305__dra     187.2         0   0.00%
C14818__extr EX_C14818__dra     0.136         0   0.00%

Secretion
---------
      Metabolite           Reaction       Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo -2.655E+05         0   0.00%
 photon680__chlo DM_photon680__chlo -6.486E+04         0   0.00%
    C00007__extr     EX_C00007__dra -1.486E+05         0   0.00%
    C00027__extr     EX_C00027__dra     -561.5         0   0.00%
    C00237__ex

In order to run GIMME or eFlux, you need to set your expression data.

In [28]:
expr = pd.read_csv('data/inputs/tpm.tsv', sep='\t') # load expression data

expr["Geneid"] = expr["Geneid"] + "_RA" # add a suffix to the geneid to avoid conflicts with the model
n_genes = expr.shape[0] # number of genes
print("Number of genes:", n_genes)
print("Number of samples:", expr.shape[1]-1)
print("Head of the expression data:")
print(expr.head())
print("\n")
print("Summary of expression data:")
print(expr.describe())

Number of genes: 11261
Number of samples: 1
Head of the expression data:
       Geneid  tpm
0  Ng00001_RA  0.0
1  Ng00002_RA  0.0
2  Ng00003_RA  0.0
3  Ng00004_RA  0.0
4  Ng00005_RA  0.0


Summary of expression data:
                tpm
count  11261.000000
mean      88.802060
std      417.656209
min        0.000000
25%        0.000000
50%        0.000000
75%       74.315983
max    12133.104496


In [29]:
identifiers = expr['Geneid'].tolist() # list of gene identifiers
conditions = ['tpm'] # list of conditions
expression = expr['tpm'].to_numpy()[:, np.newaxis] # it has to be a numpy array
set_expression = ExpressionSet(identifiers, conditions, expression)
if set_expression is not None:
    print("Expression data has been set.")

Expression data has been set.


Set the desired solver to be used. In this case the Gurobi solver will be used.

In [3]:
mewpy.solvers.set_default_solver('gurobi')
print(mewpy.solvers.get_default_solver())
my_model.model.solver = 'gurobi'

gurobi


Now it is possible to run GIMME and eFlux.

In [7]:
mewpy.solvers.set_default_solver('gurobi')
my_model.model.solver = 'glpk'
expr = pd.read_csv('data/inputs/tpm.tsv', sep='\t')
conditions = ['tpm']
my_model.gimme(expr=expr, conditions=['tpm'])

SolverError: CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.

In [20]:
my_model.eflux(expr, conditions=conditions)

TypeError: eFlux() takes from 2 to 7 positional arguments but 8 were given

In [49]:
#identifiers: list = expr['Geneid'].tolist()
#conditions: list = ['tpm']
#expression = expr['tpm'].to_numpy()[:, np.newaxis]

#set_expression = ExpressionSet(identifiers, conditions, expression)

In [ ]:
from src.integration import Integration

In [12]:
integration = Integration("data/inputs/model_ngaditana.xml")

72520 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Model loaded
Objective
1.0 e_Biomass__cytop = 1904.5048036884596

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra 7.067E+04         0   0.00%
C00009__extr EX_C00009__dra      2499         0   0.00%
C00011__extr EX_C00011__dra 9.842E+04         1 100.00%
C00059__extr EX_C00059__dra     177.3         0   0.00%
C00080__extr EX_C00080__dra      2676         0   0.00%
C00205__extr EX_C00205__dra     1E+06         0   0.00%
C00244__extr EX_C00244__dra 1.611E+04         0   0.00%
C00305__extr EX_C00305__dra     187.2         0   0.00%
C14818__extr EX_C14818__dra     0.136         0   0.00%

Secretion
---------
      Metabolite           Reaction       Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo -2.655E+05         0   0.00%
 photon680__chlo DM_photon680__chlo -6.486E+04         0   0.00%
    C00007__extr     EX_C00007__dra -1.486E+05         0   0.00%
    C00027__extr     EX_C00027__dra     -561.5         0   0.00%
    C00237__ex

In [16]:
expr = pd.read_csv('data/inputs/tpm.tsv', sep='\t')
expr["Geneid"] = expr["Geneid"] + "_RA"
condition = ['tpm']

In [17]:
mewpy.solvers.set_default_solver('gurobi')
gimme_res = integration.gimme(expr, condition)

SolverError: CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.

In [29]:
solution = gimme_res.to_dataframe()

NameError: name 'gimme_res' is not defined

In [30]:
eflux = integration.eflux(expr, "data/results", "gimme_results", condition)

TypeError: eFlux() takes from 2 to 7 positional arguments but 8 were given

NameError: name 'eflux' is not defined

NameError: name 'eflux' is not defined

In [18]:
expr = pd.read_csv('data/inputs/tpm.tsv', sep='\t')
expr["Geneid"] = expr["Geneid"] + "_RA"
condition = ['tpm']

In [23]:
mewpy.solvers.set_default_solver('gurobi')
#gimme_res = integration.gimme(expr, condition)

AttributeError: module 'mewpy.solvers' has no attribute 'get_default_ode_solver'

In [8]:
solution = gimme_res.to_dataframe()

In [ ]:
eflux = integration.eflux(expr, "data/results", "gimme_results", condition)

In [25]:
from src.integration import Integration
integration = Integration("data/inputs/model_ngaditana.xml")

72520 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Model loaded
Objective
1.0 e_Biomass__cytop = 1904.5048036884596

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra 7.067E+04         0   0.00%
C00009__extr EX_C00009__dra      2499         0   0.00%
C00011__extr EX_C00011__dra 9.842E+04         1 100.00%
C00059__extr EX_C00059__dra     177.3         0   0.00%
C00080__extr EX_C00080__dra      2676         0   0.00%
C00205__extr EX_C00205__dra     1E+06         0   0.00%
C00244__extr EX_C00244__dra 1.611E+04         0   0.00%
C00305__extr EX_C00305__dra     187.2         0   0.00%
C14818__extr EX_C14818__dra     0.136         0   0.00%

Secretion
---------
      Metabolite           Reaction       Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo -2.655E+05         0   0.00%
 photon680__chlo DM_photon680__chlo -6.486E+04         0   0.00%
    C00007__extr     EX_C00007__dra -1.486E+05         0   0.00%
    C00027__extr     EX_C00027__dra     -561.5         0   0.00%
    C00237__ex

In [9]:
eflux

objective: 0.0
Status: OPTIMAL
Constraints: OrderedDict([('R02434__chlo', (0, 1)), ('T_L_Tyrosine_V2__mitmem', (-0.0, 0.0)), ('TR0011455_PLAS__ermem', (-0.017085380116959065, 0.017085380116959065)), ('R01652__cytop', (0, 0.11094402673350041)), ('R00243__mito', (-0.02657778295295761, 0.02657778295295761)), ('PKS_C204__cytop', (0, 0.006925008153760967)), ('EX_C00188__dra', (0, 1)), ('EX_C05839__dra', (0, 1)), ('R03362__cytop', (0, 0.0)), ('R04537__cytop', (0, 0.05260277129605623)), ('R02731__cytop', (0, 0.07040678619625987)), ('TO1000117_PLAS__plas', (0, 0.01837928153717627)), ('R00010__cytop', (0, 0.020369915887641955)), ('R07396__chlo', (0, 0.0)), ('R00848__mito', (0, 0.0)), ('T_CoA___chlomem', (-1, 1)), ('R08174__cytop', (0, 0.0796892729335777)), ('TO1000157_PLAS__plas', (0, 0.0)), ('TO1000060_PLAS__plas', (0, 0.01837928153717627)), ('R12205__chlo', (0, 0.0176501860712387)), ('R07084__cytop', (0, 0.14685370925676994)), ('R07109__cytop', (0, 0.0)), ('R08183__mito', (0, 0.07968927293357

In [3]:
my_model.fva()

FVA finds the ranges of each metabolic flux at the optimum.
                               minimum       maximum
R02434__chlo                  0.000000      0.000000
T_L_Tyrosine_V2__mitmem -548343.564067      0.096624
TR0011455_PLAS__ermem         0.000000      0.000000
R01652__cytop                 0.000000  12666.539865
R00243__mito            -561792.117499  13128.334099
PKS_C204__cytop               0.000000    161.567387
EX_C00188__dra                0.000000      0.000000
EX_C05839__dra                0.000000      0.000000
R03362__cytop                 0.000000  66546.188139
R04537__cytop                 0.000000   1823.094117


AttributeError: DictList has no attribute or entry FRD7

In [4]:
my_model.pFBA()

0.0